# Timeseries
Statistics on fundamentally temporal data.

In [1]:
#| default_exp stats.timeseries

## Basic computations

In [7]:
#| export
from pathlib import Path
import polars as pl


def add_step_columns(df: pl.DataFrame,
                     *,
                     x_col:str="X",
                     y_col:str="Y") -> pl.DataFrame:
    """
    Add delta_dist and angle to the DataFrame.
    Assumes rows are already sorted by `t`.
    """
    return (
        df.with_columns(
            delta_x=pl.col(x_col).diff().fill_null(0.0),
            delta_y=pl.col(y_col).diff().fill_null(0.0),
        )
        .with_columns(
            delta_dist=(pl.col("delta_x") ** 2 + pl.col("delta_y") ** 2).sqrt(),
            angle=pl.arctan2(pl.col("delta_y"), pl.col("delta_x")),
        )
        .drop(["delta_x", "delta_y"])
    )


### Prep code

The following code is needed to load the data and get it to a working state. This will be beautified soon.
Ignore during demo

In [ ]:
from RevChem.data_export import read_chunks_from_json
from RevChem.timeseries_segmentation import join_chunks_as_segments

# Loading JSON  store of the RealEye and Tobii pairs in a format that's fast to read: 4 seconds for 35 trials.
# NOTE: this is ALL of the data from the study
associated_tobii_re_sequences = read_chunks_from_json(
    Path("~/dev/RevChemData/2025-07-17-python-outputs/202507171142-matches-with-TCA.json.gz").expanduser(),
)

In [ ]:
# rather than having all of the data just by trial, we've now got per-trial, per-stimulu blocks to work
trials_by_stimuli = join_chunks_as_segments(associated_tobii_re_sequences)

In [19]:
#| export 
# TODO(stephen): move to the correct module -> data.segmentation
from dataclasses import dataclass # FIXME: dataclass versus namedtuple. Former is clearer, latter is faster.
from RevChem.timeseries_segmentation import AssociatedTrialSegements

@dataclass
class TypedSegment:
    re: pl.DataFrame # realeye
    tobii: pl.DataFrame # tobii

@dataclass
class AssociatedTrialRetyped:
    trial_name: str
    segments: list[TypedSegment]

def split_segment(segment_df: pl.DataFrame) -> TypedSegment:
    "Split the 5 columns (timestamp, X, Y, X_re, Y_re) two ways, into RealEye and Tobii parts, sharing time"
    new_names = {"timestamp": "t"}
    return TypedSegment(
        segment_df.select("timestamp", "X_re", "Y_re").rename(new_names |{"X_re": "X", "Y_re": "Y"}),
        segment_df.select("timestamp", "X", "Y").rename(new_names)
    )

def retype_associated_data(data: AssociatedTrialSegements) -> AssociatedTrialRetyped:
    return AssociatedTrialRetyped(
        data.trial_name_or_id,
        [split_segment(seg) for seg in data.segments]
    )

In [20]:
retype_associated_data(trials_by_stimuli[0])

AssociatedTrialRetyped(trial_name='2025-03-07-Cyndaquil', segments=[TypedSegment(re=shape: (117, 3)
┌────────────────────────────┬──────┬─────┐
│ t                          ┆ X    ┆ Y   │
│ ---                        ┆ ---  ┆ --- │
│ datetime[μs]               ┆ i32  ┆ i32 │
╞════════════════════════════╪══════╪═════╡
│ 2025-03-07 18:47:07.934088 ┆ 956  ┆ 547 │
│ 2025-03-07 18:47:07.942421 ┆ 956  ┆ 547 │
│ 2025-03-07 18:47:07.950754 ┆ 956  ┆ 547 │
│ 2025-03-07 18:47:07.959088 ┆ 892  ┆ 514 │
│ 2025-03-07 18:47:07.967421 ┆ 892  ┆ 514 │
│ …                          ┆ …    ┆ …   │
│ 2025-03-07 18:47:08.867402 ┆ 1064 ┆ 624 │
│ 2025-03-07 18:47:08.875736 ┆ 1064 ┆ 624 │
│ 2025-03-07 18:47:08.884066 ┆ 1064 ┆ 624 │
│ 2025-03-07 18:47:08.892397 ┆ 1064 ┆ 624 │
│ 2025-03-07 18:47:08.900731 ┆ 1064 ┆ 624 │
└────────────────────────────┴──────┴─────┘, tobii=shape: (117, 3)
┌────────────────────────────┬──────┬──────┐
│ t                          ┆ X    ┆ Y    │
│ ---                        ┆ ---  ┆ -

### Basic computations demo

In [21]:
trial_data = retype_associated_data(trials_by_stimuli[0])
add_step_columns(trial_data.segments[0].re)

t,X,Y,delta_dist,angle
datetime[μs],i32,i32,f64,f64
2025-03-07 18:47:07.934088,956,547,0.0,0.0
2025-03-07 18:47:07.942421,956,547,0.0,0.0
2025-03-07 18:47:07.950754,956,547,0.0,0.0
2025-03-07 18:47:07.959088,892,514,72.006944,-2.665523
2025-03-07 18:47:07.967421,892,514,0.0,0.0
…,…,…,…,…
2025-03-07 18:47:08.867402,1064,624,54.671748,0.876058
2025-03-07 18:47:08.875736,1064,624,0.0,0.0
2025-03-07 18:47:08.884066,1064,624,0.0,0.0


In [22]:
add_step_columns(trial_data.segments[0].tobii)

t,X,Y,delta_dist,angle
datetime[μs],i32,i32,f64,f64
2025-03-07 18:47:07.934088,912,684,0.0,0.0
2025-03-07 18:47:07.942421,914,684,2.0,0.0
2025-03-07 18:47:07.950754,915,684,1.0,0.0
2025-03-07 18:47:07.959088,915,692,8.0,1.570796
2025-03-07 18:47:07.967421,911,691,4.123106,-2.896614
…,…,…,…,…
2025-03-07 18:47:08.867402,1128,588,71.8401,2.464678
2025-03-07 18:47:08.875736,null,null,0.0,0.0
2025-03-07 18:47:08.884066,null,null,0.0,0.0


### Summary statistics

#### Aggregate summary statistics

In [25]:
#| export
def global_summary(df: pl.DataFrame,
                   *,
                   x_col: str = "X",
                   y_col: str = "Y") -> pl.DataFrame:
    """
    Return a single-row DataFrame with global trajectory statistics.
    """
    return df.select(
        total_distance=pl.col("delta_dist").sum(),
        avg_speed=pl.col("delta_dist").mean(),
        std_speed=pl.col("delta_dist").std(),
        min_speed=pl.col("delta_dist").min(),
        max_speed=pl.col("delta_dist").max(),
        net_displacement=(
            (pl.col(x_col).last() - pl.col(x_col).first()) ** 2
            + (pl.col(y_col).last() - pl.col(y_col).first()) ** 2
        ).sqrt(),
        total_time=pl.col("t").last() - pl.col("t").first(),
        sinuosity=(
            pl.col("delta_dist").sum()
            / (
                (pl.col(x_col).last() - pl.col(x_col).first()) ** 2
                + (pl.col(y_col).last() - pl.col(y_col).first()) ** 2
            ).sqrt()
        ),
    )


In [23]:
global_summary(
    add_step_columns(trial_data.segments[0].re)
)

total_distance,avg_speed,std_speed,min_speed,max_speed,net_displacement,total_time,sinuosity
f64,f64,f64,f64,f64,f64,duration[μs],f64
1622.784676,13.869954,30.085088,0.0,122.262014,132.638607,966643µs,12.234633


In [24]:
global_summary(
    add_step_columns(trial_data.segments[0].tobii)
)

total_distance,avg_speed,std_speed,min_speed,max_speed,net_displacement,total_time,sinuosity
f64,f64,f64,f64,f64,f64,duration[μs],f64
995.835688,8.511416,14.798074,0.0,71.8401,null,966643µs,null


#### Rolling summary statistics

In [29]:
#| export
def rolling_summary(df: pl.DataFrame, window: int) -> pl.DataFrame:
    """
    Rolling statistics on speed and angle.
    """
    return (
        df.with_columns(
            roll_speed_mean=pl.col("delta_dist")
            .rolling_mean(window_size=window, min_samples=1)
            .alias("speed_mean"),
            roll_speed_std=pl.col("delta_dist")
            .rolling_std(window_size=window, min_samples=1)
            .alias("speed_std"),
            roll_angle_mean=pl.col("angle")
            .rolling_mean(window_size=window, min_samples=1)
            .alias("angle_mean"),
            roll_angle_std=pl.col("angle")
            .rolling_std(window_size=window, min_samples=1)
            .alias("angle_std"),
        )
        # Optionally drop the helper columns here if you only want the stats
    )

In [30]:
rolling_summary(
    add_step_columns(trial_data.segments[0].re),
    window=100
)

t,X,Y,delta_dist,angle,roll_speed_mean,roll_speed_std,roll_angle_mean,roll_angle_std
datetime[μs],i32,i32,f64,f64,f64,f64,f64,f64
2025-03-07 18:47:07.934088,956,547,0.0,0.0,0.0,null,0.0,null
2025-03-07 18:47:07.942421,956,547,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-07 18:47:07.950754,956,547,0.0,0.0,0.0,0.0,0.0,0.0
2025-03-07 18:47:07.959088,892,514,72.006944,-2.665523,18.001736,36.003472,-0.666381,1.332762
2025-03-07 18:47:07.967421,892,514,0.0,0.0,14.401389,32.202484,-0.533105,1.192058
…,…,…,…,…,…,…,…,…
2025-03-07 18:47:08.867402,1064,624,54.671748,0.876058,14.905072,31.257627,-0.057523,0.930543
2025-03-07 18:47:08.875736,1064,624,0.0,0.0,14.905072,31.257627,-0.057523,0.930543
2025-03-07 18:47:08.884066,1064,624,0.0,0.0,14.905072,31.257627,-0.057523,0.930543


In [31]:
rolling_summary(
    add_step_columns(trial_data.segments[0].tobii),
    window=100
)

t,X,Y,delta_dist,angle,roll_speed_mean,roll_speed_std,roll_angle_mean,roll_angle_std
datetime[μs],i32,i32,f64,f64,f64,f64,f64,f64
2025-03-07 18:47:07.934088,912,684,0.0,0.0,0.0,null,0.0,null
2025-03-07 18:47:07.942421,914,684,2.0,0.0,1.0,1.414214,0.0,0.0
2025-03-07 18:47:07.950754,915,684,1.0,0.0,1.0,1.0,0.0,0.0
2025-03-07 18:47:07.959088,915,692,8.0,1.570796,2.75,3.593976,0.392699,0.785398
2025-03-07 18:47:07.967421,911,691,4.123106,-2.896614,3.024621,3.172473,-0.265164,1.620665
…,…,…,…,…,…,…,…,…
2025-03-07 18:47:08.867402,1128,588,71.8401,2.464678,9.532174,15.762045,0.294578,1.664961
2025-03-07 18:47:08.875736,null,null,0.0,0.0,9.482174,15.784482,0.285305,1.663983
2025-03-07 18:47:08.884066,null,null,0.0,0.0,9.440943,15.803995,0.298563,1.656281


## Eye-tracking Statistics
> To better show the power of a bottom-up approach, we have found a way of programmatically redefining fixations and corresponding finding AOIs as such.

In [34]:
#| export
def fixation_segments(
    df: pl.DataFrame,
    radius: float,
    *,
    x: str = "x",
    y: str = "y",
    t: str = "t",
    min_fixation_time: float | None = None,
    min_n_points: int = 1,
) -> pl.DataFrame:
    """
    Detect contiguous *fixation* segments in an eye-tracking trace.

    A fixation starts when the gaze stays **within `radius` units** from the
    *anchor* point of that fixation for at least `min_fixation_time`
    seconds (or `min_n_points` samples).  While the anchor is fixed,
    every new point is tested against this anchor.  As soon as a point
    exceeds the radius, the current fixation ends and a new anchor is set
    at that point.

    Parameters
    ----------
    df : pl.DataFrame
        Must contain the columns `x`, `y`, and `t` (units are your own).
    radius : float
        Spatial tolerance (in the same units as x, y).
    x, y, t : str
        Column names for the spatial and temporal coordinates.
    min_fixation_time : float, optional
        Minimum duration (in the same units as `t`) for a segment to be
        retained.  If None, `min_n_points` is the only filter.
    min_n_points : int, default 1
        Minimum number of consecutive points inside the radius.

    Returns
    -------
    pl.DataFrame
        One row per fixation segment with columns:
        fixation_id   – unique id per fixation (monotonic)
        t_start    – first timestamp of the fixation
        t_end      – last timestamp of the fixation
        x_anchor   – x-coordinate of the anchor point
        y_anchor   – y-coordinate of the anchor point
        duration   – t_end - t_start
        n_points   – number of rows in the fixation
    """
    if df.is_empty():
        return pl.DataFrame(
            {
                "fixation_id": pl.Series(dtype=pl.Int64),
                "t_start": pl.Series(dtype=pl.Int64),
                "t_end": pl.Series(dtype=pl.Int64),
                "x_anchor": pl.Series(dtype=pl.Float64),
                "y_anchor": pl.Series(dtype=pl.Float64),
                "n_points": pl.Series(dtype=pl.Int64),
                "duration": pl.Series(dtype=pl.Int64),
            }
        )

    # Ensure ascending order
    df = df.sort(t)
    df = df.with_row_index(name="i")

    # Convert to Python dicts for iteration
    rows = df.to_dicts()
    fixations = []
    if not rows:
        return pl.DataFrame(fixations)

    # Initial anchor point
    current_fix_start_idx = 0
    anchor_x = rows[0][x]
    anchor_y = rows[0][y]

    for i in range(1, len(rows)):
        dist = ((rows[i][x] - anchor_x) ** 2 + (rows[i][y] - anchor_y) ** 2) ** 0.5
        if dist > radius:
            # End of fixation, record it
            fixations.append(
                {
                    "i_start": rows[current_fix_start_idx]["i"],
                    "i_end": rows[i - 1]["i"],
                    "x_anchor": anchor_x,
                    "y_anchor": anchor_y,
                }
            )
            # New fixation starts at current point
            current_fix_start_idx = i
            anchor_x = rows[i][x]
            anchor_y = rows[i][y]

    # Add the last fixation
    fixations.append(
        {
            "i_start": rows[current_fix_start_idx]["i"],
            "i_end": rows[-1]["i"],
            "x_anchor": anchor_x,
            "y_anchor": anchor_y,
        }
    )

    # Convert back to polars DataFrame
    fix_df = pl.from_dicts(fixations)

    # Join with original data to get times and counts
    fix_df = fix_df.join(
        df.select(["i", t]).rename({"i": "i_start", t: "t_start"}),
        on="i_start",
    ).join(
        df.select(["i", t]).rename({"i": "i_end", t: "t_end"}),
        on="i_end",
    )

    # Calculate n_points and duration
    fix_df = fix_df.with_columns(
        n_points=pl.col("i_end") - pl.col("i_start") + 1,
        duration=pl.col("t_end") - pl.col("t_start"),
    ).drop(["i_start", "i_end"])

    # Apply filters
    if min_fixation_time is not None:
        fix_df = fix_df.filter(pl.col("duration") >= min_fixation_time)
    fix_df = fix_df.filter(pl.col("n_points") >= min_n_points)

    # Finalize
    return fix_df.with_row_index(name="fixation_id").select(
        [
            "fixation_id",
            "t_start",
            "t_end",
            "x_anchor",
            "y_anchor",
            "n_points",
            "duration",
        ]
    )


### Fixations demo

In [37]:
demo_data_re_5px_fixations = fixation_segments(
    trial_data.segments[0].re,
    radius=5.0,
    x="X",
    y="X"
)
demo_data_re_5px_fixations

fixation_id,t_start,t_end,x_anchor,y_anchor,n_points,duration
u32,datetime[μs],datetime[μs],i64,i64,i64,duration[μs]
0,2025-03-07 18:47:07.934088,2025-03-07 18:47:07.950754,956,956,3,16666µs
1,2025-03-07 18:47:07.959088,2025-03-07 18:47:08.017421,892,892,8,58333µs
2,2025-03-07 18:47:08.025754,2025-03-07 18:47:08.050754,952,952,4,25ms
3,2025-03-07 18:47:08.059086,2025-03-07 18:47:08.084084,1000,1000,4,24998µs
4,2025-03-07 18:47:08.092418,2025-03-07 18:47:08.109084,1090,1090,3,16666µs
…,…,…,…,…,…,…
21,2025-03-07 18:47:08.734070,2025-03-07 18:47:08.759070,920,920,4,25ms
22,2025-03-07 18:47:08.767402,2025-03-07 18:47:08.792402,973,973,4,25ms
23,2025-03-07 18:47:08.800736,2025-03-07 18:47:08.825736,911,911,4,25ms


In [38]:
fixation_segments(
    trial_data.segments[0].re,
    radius=1.0,
    x="X",
    y="X"
)

fixation_id,t_start,t_end,x_anchor,y_anchor,n_points,duration
u32,datetime[μs],datetime[μs],i64,i64,i64,duration[μs]
0,2025-03-07 18:47:07.934088,2025-03-07 18:47:07.950754,956,956,3,16666µs
1,2025-03-07 18:47:07.959088,2025-03-07 18:47:07.984088,892,892,4,25ms
2,2025-03-07 18:47:07.992421,2025-03-07 18:47:08.017421,894,894,4,25ms
3,2025-03-07 18:47:08.025754,2025-03-07 18:47:08.050754,952,952,4,25ms
4,2025-03-07 18:47:08.059086,2025-03-07 18:47:08.084084,1000,1000,4,24998µs
…,…,…,…,…,…,…
23,2025-03-07 18:47:08.734070,2025-03-07 18:47:08.759070,920,920,4,25ms
24,2025-03-07 18:47:08.767402,2025-03-07 18:47:08.792402,973,973,4,25ms
25,2025-03-07 18:47:08.800736,2025-03-07 18:47:08.825736,911,911,4,25ms


In [40]:
fixation_segments(
    trial_data.segments[0].tobii.drop_nulls(),
    radius=5.0,
    x="X",
    y="X"
)

fixation_id,t_start,t_end,x_anchor,y_anchor,n_points,duration
u32,datetime[μs],datetime[μs],i64,i64,i64,duration[μs]
0,2025-03-07 18:47:07.934088,2025-03-07 18:47:08.000754,912,912,9,66666µs
1,2025-03-07 18:47:08.009088,2025-03-07 18:47:08.084084,917,917,10,74996µs
2,2025-03-07 18:47:08.092418,2025-03-07 18:47:08.142418,913,913,7,50ms
3,2025-03-07 18:47:08.150751,2025-03-07 18:47:08.150751,926,926,1,0µs
4,2025-03-07 18:47:08.159084,2025-03-07 18:47:08.225750,937,937,9,66666µs
…,…,…,…,…,…,…
27,2025-03-07 18:47:08.817402,2025-03-07 18:47:08.834069,1165,1165,3,16667µs
28,2025-03-07 18:47:08.842402,2025-03-07 18:47:08.842402,1170,1170,1,0µs
29,2025-03-07 18:47:08.850736,2025-03-07 18:47:08.850736,1163,1163,1,0µs


In [41]:
fixation_segments(
    trial_data.segments[0].tobii.drop_nulls(),
    radius=1.0,
    x="X",
    y="X"
)

fixation_id,t_start,t_end,x_anchor,y_anchor,n_points,duration
u32,datetime[μs],datetime[μs],i64,i64,i64,duration[μs]
0,2025-03-07 18:47:07.934088,2025-03-07 18:47:07.934088,912,912,1,0µs
1,2025-03-07 18:47:07.942421,2025-03-07 18:47:07.942421,914,914,1,0µs
2,2025-03-07 18:47:07.950754,2025-03-07 18:47:07.959088,915,915,2,8334µs
3,2025-03-07 18:47:07.967421,2025-03-07 18:47:07.967421,911,911,1,0µs
4,2025-03-07 18:47:07.975754,2025-03-07 18:47:07.975754,912,912,1,0µs
…,…,…,…,…,…,…
89,2025-03-07 18:47:08.834069,2025-03-07 18:47:08.834069,1162,1162,1,0µs
90,2025-03-07 18:47:08.842402,2025-03-07 18:47:08.842402,1170,1170,1,0µs
91,2025-03-07 18:47:08.850736,2025-03-07 18:47:08.850736,1163,1163,1,0µs


### Clustering for AOIs
> Expert-defined and subjective AOIs are a source for much discussion in eye-tracking research.
> This approach lets the data speak for itself, using the prior that the subject may not be able to accurately describe what they were looking at, but the data can.
> 
> NOTE OF CAUTION: this has a meaningful interplay with the definitions of the fixations supplied. Thus, given examples may have erroneous assumptions.

In [43]:
#| export
def aoi_from_fixations(
    fixations: pl.DataFrame, eps: float, min_samples: int
) -> pl.DataFrame:
    """
    Identify Areas of Interest (AOIs) from fixation data using DBSCAN.

    Parameters
    ----------
    fixations : pl.DataFrame
        A DataFrame of fixation data, as returned by `fixation_segments`.
        Must contain `x_anchor` and `y_anchor` columns.
    eps : float
        The maximum distance between two samples for one to be considered
        as in the neighborhood of the other.
    min_samples : int
        The number of samples in a neighborhood for a point to be
        considered as a core point.

    Returns
    -------
    pl.DataFrame
        A DataFrame with AOI statistics, including the number of fixations
        in each AOI, the total duration, and the bounding box of the AOI.
    """
    try:
        from sklearn.cluster import DBSCAN

        failed_import = False
    except ImportError:
        failed_import = True

    if fixations.is_empty() or failed_import:
        return pl.DataFrame(
            {
                "aoi_id": pl.Series(dtype=pl.Int64),
                "n_fixations": pl.Series(dtype=pl.Int64),
                "total_duration": pl.Series(dtype=pl.Int64),
                "x_min": pl.Series(dtype=pl.Float64),
                "y_min": pl.Series(dtype=pl.Float64),
                "x_max": pl.Series(dtype=pl.Float64),
                "y_max": pl.Series(dtype=pl.Float64),
            }
        )

    # Extract anchor points for clustering
    coords = fixations.select(["x_anchor", "y_anchor"]).to_numpy()

    # Perform DBSCAN clustering
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
    labels = db.labels_

    # Add cluster labels to fixations DataFrame
    fixations = fixations.with_columns(aoi_id=pl.Series(labels))

    # Filter out noise points (label -1)
    aois = fixations.filter(pl.col("aoi_id") != -1)

    # Aggregate statistics per AOI
    aoi_stats = aois.group_by("aoi_id").agg(
        n_fixations=pl.len(),
        total_duration=pl.col("duration").sum(),
        x_min=pl.col("x_anchor").min(),
        y_min=pl.col("y_anchor").min(),
        x_max=pl.col("x_anchor").max(),
        y_max=pl.col("y_anchor").max(),
    )

    return aoi_stats.sort("aoi_id")

In [45]:
aoi_from_fixations(
    fixation_segments(
        trial_data.segments[0].tobii.drop_nulls(), radius=5.0, x="X", y="X"
    ),
    eps=30,
    min_samples=3
)

aoi_id,n_fixations,total_duration,x_min,y_min,x_max,y_max
i64,u32,duration[μs],i64,i64,i64,i64
0,6,324994µs,912,912,941,941
1,23,349992µs,1128,1128,1257,1257
